“E-Commerce Transaction Analytics” (Using Online Retail Dataset)

🎯 Scenario / Story
You are a data analyst at ShopWave, an e-commerce platform in the UK. ShopWave logs every transaction (order) made in their store. Each record has:

Invoice number  
Stock code / product ID  
Description   
Quantity    
Invoice date   
Unit price   
Customer ID   
Country

Your manager wants you to clean the data, fix inconsistencies, derive business metrics, and explore relationships. The cleaned data should enable deeper dashboards later.

Tasks & Steps    
Below is a full pipeline of tasks that students should perform. Each task corresponds to one or more of the functions/techniques you already taught.   
Load & Inspect   
Read the CSV (e.g. OnlineRetail.csv)   
Display head(),   
 tail(),  
   info(),   
 shape   
Check dtypes    
Missing Data Handling   
Use isnull().sum() to find missing values (e.g. some Description or Customer ID missing)   
Drop rows that have missing Customer ID (because you can’t tie to a customer)       
Fill missing Description or other columns using a placeholder (e.g. "Unknown")      
Date Parsing / Format Correction
The InvoiceDate column might be string. Convert using pd.to_datetime(..., format='mixed')     
Drop records where parsing failed (if any)
Detect & Fix Bad or Outlier Data     
Some transactions might have negative quantity (returns) — detect        those and decide whether to treat as returns or drop
Some unit prices might be zero or negative — fix/clean these (e.g. drop or adjust)     
Duplicates     
Use duplicated() to find duplicated invoice-item entries    
Drop duplicates    


In [ ]:
pip install pandas

In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv("/content/OnlineRetail.csv", encoding='latin1')
df=pd.DataFrame(data)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [ ]:
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
425161,C573306,22796,PHOTO FRAME 3 CLASSIC HANGING,-1.0,10/28/2011 16:15,9.95,17837.0,United Kingdom
425162,C573306,82483,WOOD 2 DRAWER CABINET WHITE FINISH,-1.0,10/28/2011 16:15,6.95,17837.0,United Kingdom
425163,573307,POST,POSTAGE,1.0,10/28/2011 16:24,11.27,NaN,United Kingdom
425164,573308,23051,RECYCLED ACAPULCO MAT BLUE,1.0,10/28/2011 16:32,8.25,NaN,United Kingdom
425165,5733,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(df.shape)
print(df.info())

(425166, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425166 entries, 0 to 425165
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    425166 non-null  object 
 1   StockCode    425165 non-null  object 
 2   Description  423804 non-null  object 
 3   Quantity     425165 non-null  float64
 4   InvoiceDate  425165 non-null  object 
 5   UnitPrice    425165 non-null  float64
 6   CustomerID   318653 non-null  float64
 7   Country      425165 non-null  object 
dtypes: float64(3), object(5)
memory usage: 26.0+ MB
None


In [ ]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,1
Description,1362
Quantity,1
InvoiceDate,1
UnitPrice,1
CustomerID,106513
Country,1


In [ ]:
df = df.dropna(subset=['CustomerID'])
df['Description'] = df['Description'].fillna("Unknown")

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
df = df.dropna(subset=['InvoiceDate'])

In [ ]:
print(df[df['Quantity'] < 0].shape[0])
returns = df[df['Quantity'] < 0]
df = df[df['Quantity'] > 0]

0


In [ ]:
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

Duplicate rows: 3490


In [ ]:
df = df.drop_duplicates()

In [ ]:
df.to_csv("OnlineRetail_Cleaned.csv", index=False)

Filtering & Sorting    
Filter transactions where Quantity > 1000 (suspicious outliers)
Sort by UnitPrice * Quantity descending (i.e. highest revenue orders)
Merge / Enrich Data        
Create a small custom DataFrame product_info with columns StockCode, Category, Supplier                  
Convert its date (if needed) and merge with the main dataset on StockCode (left join)               
Group & Aggregation               
Group by Country and compute total revenue, mean revenue per order, and count of orders            
Create a new column Revenue = Quantity * UnitPrice         
Use pd.cut() to bucket UnitPrice (e.g. cheap, medium, expensive) and then group by that price range to see how many purchases and average revenue                
Correlation / Relationships              
Use .corr() on numeric columns (Quantity, UnitPrice, Revenue)        
Interpret which factors are strongly related (e.g. high quantity often correlates with higher revenue)         
Rename & String Operations                
Rename StockCode to Product_ID, UnitPrice to Price               
Use .str methods on Description — e.g., .str.lower(), .str.contains("glass"), etc.               
Filter products whose description contains certain keywords

In [ ]:

filtered_df = df[df['Quantity'] > 1000]
print("Transactions with Quantity > 1000:")
display(filtered_df.head())


Transactions with Quantity > 1000:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
4850,536809,84950,ASSORTED COLOUR T-LIGHT HOLDER,1824.0,2010-12-02 16:48:00,0.55,15299.0,United Kingdom,1003.20
4945,536830,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,2880.0,2010-12-02 17:38:00,0.18,16754.0,United Kingdom,518.40
4946,536830,21915,RED HARMONICA IN BOX,1400.0,2010-12-02 17:38:00,1.06,16754.0,United Kingdom,1484.00
6365,536890,17084R,ASSORTED INCENSE PACK,1440.0,2010-12-03 11:48:00,0.16,14156.0,EIRE,230.40
16435,537659,22189,CREAM HEART CARD HOLDER,1008.0,2010-12-07 16:43:00,2.31,18102.0,United Kingdom,2328.48


In [ ]:
df['Revenue'] = df['UnitPrice'] * df['Quantity']
sorted_df = df.sort_values(by='Revenue', ascending=False)
print("\nDataFrame sorted by Revenue (descending):")
display(sorted_df.head())


DataFrame sorted by Revenue (descending):


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215.0,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.60
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60.0,2011-06-10 15:28:00,649.50,15098.0,United Kingdom,38970.00
173382,551697,POST,POSTAGE,1.0,2011-05-03 13:46:00,8142.75,16029.0,United Kingdom,8142.75
348325,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412.0,2011-09-20 11:05:00,5.06,17450.0,United Kingdom,7144.72
160546,550461,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114.0,2011-04-18 13:20:00,2.10,15749.0,United Kingdom,6539.40


In [ ]:

product_info = pd.DataFrame({
    'StockCode': ['85123A', '71053', '84406B', '84029G', '84029E'],
    'Category': ['Home Decor', 'Home Decor', 'Home Decor', 'Textiles', 'Textiles'],
    'Supplier': ['Supplier A', 'Supplier B', 'Supplier A', 'Supplier C', 'Supplier C']
})


In [ ]:
merged_df = pd.merge(df, product_info, on='StockCode', how='left')

In [ ]:
print("DataFrame after merging with product_info:")
display(merged_df.head())

DataFrame after merging with product_info:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Category,Supplier
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,Home Decor,Supplier A
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Home Decor,Supplier B
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,Home Decor,Supplier A
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Textiles,Supplier C
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Textiles,Supplier C


Descriptive Statistics              
Use .describe() on numeric columns                   
Use .value_counts() on Country or Category                 
Advanced Filtering / Querying                 
Use .query() to find orders where Revenue > 1000 and Country == 'Germany'                      
Use .between() to capture Quantity between 10 and 100                    
Export Cleaned Data              
Save the final cleaned & enriched dataset to CSV / Excel


In [ ]:

correlation_matrix = df[['Quantity', 'Price', 'Revenue']].corr()

print("\nCorrelation matrix of numeric columns:")
display(correlation_matrix)


Correlation matrix of numeric columns:


,Quantity,Price,Revenue
Quantity,1.000000,-0.005840,0.845486
Price,-0.005840,1.000000,0.160042
Revenue,0.845486,0.160042,1.000000


In [ ]:
print(f"\nRows with UnitPrice <= 0 before removal: {df[df['Price'] <= 0].shape[0]}")
df = df[df['Price'] > 0]
print(f"Rows remaining after removing UnitPrice <= 0: {df.shape[0]}")
display(df.head())


Rows with UnitPrice <= 0 before removal: 0
Rows remaining after removing UnitPrice <= 0: 307660


,InvoiceNo,Product_ID,Description,Quantity,InvoiceDate,Price,CustomerID,Country,Revenue,Price_Range
0,536365,85123A,white hanging heart t-light holder,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,cheap
1,536365,71053,white metal lantern,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,cheap
2,536365,84406B,cream cupid hearts coat hanger,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,cheap
3,536365,84029G,knitted union flag hot water bottle,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,cheap
4,536365,84029E,red woolly hottie white heart.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,cheap


In [ ]:
df = df.rename(columns={'StockCode': 'Product_ID', 'UnitPrice': 'Price'})
print("\nDataFrame after renaming columns:")
display(df.head())


DataFrame after renaming columns:


,InvoiceNo,Product_ID,Description,Quantity,InvoiceDate,Price,CustomerID,Country,Revenue,Price_Range
0,536365,85123A,white hanging heart t-light holder,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,cheap
1,536365,71053,white metal lantern,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,cheap
2,536365,84406B,cream cupid hearts coat hanger,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,cheap
3,536365,84029G,knitted union flag hot water bottle,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,cheap
4,536365,84029E,red woolly hottie white heart.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,cheap


In [ ]:
df['Description'] = df['Description'].str.lower()
print("\nDataFrame after converting Description to lowercase:")
display(df.head())



DataFrame after converting Description to lowercase:


,InvoiceNo,Product_ID,Description,Quantity,InvoiceDate,Price,CustomerID,Country,Revenue,Price_Range
0,536365,85123A,white hanging heart t-light holder,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,cheap
1,536365,71053,white metal lantern,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,cheap
2,536365,84406B,cream cupid hearts coat hanger,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,cheap
3,536365,84029G,knitted union flag hot water bottle,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,cheap
4,536365,84029E,red woolly hottie white heart.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,cheap


In [ ]:
glass_products = df[df['Description'].str.contains("glass", na=False)]
print("\nProducts with 'glass' in their description:")
display(glass_products.head())


Products with 'glass' in their description:


,InvoiceNo,Product_ID,Description,Quantity,InvoiceDate,Price,CustomerID,Country,Revenue,Price_Range
6,536365,21730,glass star frosted t-light holder,6.0,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,25.50,cheap
64,536373,21730,glass star frosted t-light holder,6.0,2010-12-01 09:02:00,4.25,17850.0,United Kingdom,25.50,cheap
81,536375,21730,glass star frosted t-light holder,6.0,2010-12-01 09:32:00,4.25,17850.0,United Kingdom,25.50,cheap
138,536381,22778,glass cloche small,3.0,2010-12-01 09:41:00,3.95,15311.0,United Kingdom,11.85,cheap
148,536382,22798,antique glass dressing table pot,8.0,2010-12-01 09:45:00,2.95,16098.0,United Kingdom,23.60,cheap


In [ ]:
bins = [0, 5, 20, df['Price'].max()]
labels = ['cheap', 'medium', 'expensive']
df['Price_Range'] = pd.cut(df['Price'], bins=bins, labels=labels, right=False)


In [ ]:
price_range_agg = df.groupby('Price_Range').agg(
    number_of_purchases=('InvoiceNo', 'count'),
    average_revenue=('Revenue', 'mean')
).reset_index()
print("\nAggregated data by Price Range:")
display(price_range_agg)


Aggregated data by Price Range:


/tmp/ipython-input-3034270804.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_range_agg = df.groupby('Price_Range').agg(


,Price_Range,number_of_purchases,average_revenue
0,cheap,266163,21.109680
1,medium,40599,33.039401
2,expensive,897,197.585340


In [ ]:
country_agg = df.groupby('Country').agg(
    total_revenue=('Revenue', 'sum'),
    mean_revenue_per_order=('Revenue', 'mean'),
    order_count=('InvoiceNo', 'nunique')
).reset_index()

print("\nAggregated data by Country:")
display(country_agg.head())


Aggregated data by Country:


,Country,total_revenue,mean_revenue_per_order,order_count
0,Australia,131211.09,115.097447,51
1,Austria,8185.70,25.500623,14
2,Bahrain,548.40,32.258824,2
3,Belgium,32700.94,19.842803,81
4,Brazil,1143.60,35.737500,1


In [ ]:

display(df.describe())

,Quantity,InvoiceDate,Price,CustomerID,Revenue
count,307660.000000,307660,307660.000000,307660.00000,307660.000000
mean,13.533124,2011-06-04 10:38:00.365663488,3.196644,15277.32453,23.224856
min,1.000000,2010-12-01 08:26:00,0.001000,12346.00000,0.001000
25%,2.000000,2011-03-14 12:58:00,1.250000,13895.00000,5.040000
50%,6.000000,2011-06-12 12:00:00,1.950000,15145.00000,12.750000
75%,12.000000,2011-09-07 13:06:00,3.750000,16779.00000,19.800000
max,74215.000000,2011-10-28 16:13:00,8142.750000,18287.00000,77183.600000
std,141.235130,NaN,24.292803,1714.46753,174.302207


In [ ]:
display(df['Country'].value_counts().head())
display(df['Product_ID'].value_counts().head())

,count
Country,
United Kingdom,271792
Germany,7577
France,6470
EIRE,5840
Netherlands,2048


,count
Product_ID,
85123A,1704
22423,1482
85099B,1346
47566,1292
84879,1130


In [ ]:

query_result = df.query("Revenue > 1000 and Country == 'Germany'")
display(query_result.head())

,InvoiceNo,Product_ID,Description,Quantity,InvoiceDate,Price,CustomerID,Country,Revenue,Price_Range


In [ ]:
between_quantity = df[df['Quantity'].between(10, 100)]
display(between_quantity.head())

,InvoiceNo,Product_ID,Description,Quantity,InvoiceDate,Price,CustomerID,Country,Revenue,Price_Range
9,536367,84879,assorted colour bird ornament,32.0,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08,cheap
26,536370,22728,alarm clock bakelike pink,24.0,2010-12-01 08:45:00,3.75,12583.0,France,90.00,cheap
27,536370,22727,alarm clock bakelike red,24.0,2010-12-01 08:45:00,3.75,12583.0,France,90.00,cheap
28,536370,22726,alarm clock bakelike green,12.0,2010-12-01 08:45:00,3.75,12583.0,France,45.00,cheap
29,536370,21724,panda and bunnies sticker sheet,12.0,2010-12-01 08:45:00,0.85,12583.0,France,10.20,cheap


In [ ]:

df.to_csv("OnlineRetail_Fully_Cleaned.csv", index=False)